In [13]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from tensorflow.keras.utils import pad_sequences
from tensorflow.keras.models import Sequential
from transformers import DistilBertTokenizer, create_optimizer
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, SimpleRNN, Flatten, Masking, Conv1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

In [2]:
data = pd.read_csv('Notebooks/data/extra.csv')

In [3]:
X = data['text']
y = data['isfake']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.1, stratify=None,  random_state=42)

In [6]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
def tokenize(texts):
    return tokenizer(texts, padding=True, truncation=True, return_tensors='tf', max_length=512)

train_encodings = tokenize(X_train.tolist())
test_encodings = tokenize(X_test.tolist())

2023-12-12 14:28:23.565449: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-12-12 14:28:24.352663: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 402 MB memory:  -> device: 0, name: NVIDIA L4, pci bus id: 0000:00:03.0, compute capability: 8.9


In [7]:
train= np.array(train_encodings['input_ids'])
train1 = np.expand_dims(train, axis=2)

In [15]:
model_rnn = Sequential([
    Masking(mask_value=0, input_shape=(512,1)),
    Conv1D(128, 5, activation='relu'), ## THIS IS THE RECURRENT LAYER
    GlobalMaxPooling1D(),
    Dense(10, activation='relu'),
    Dense(1, activation= 'sigmoid') ## final layer for prediction, hence only one node
])

## compile -- add optim, add loss, add metrics
model_rnn.compile(optimizer = 'adam', loss = ['binary_crossentropy'], metrics = ['accuracy'])

es = EarlyStopping(patience=10, restore_best_weights=True)

history_rnn = model_rnn.fit(train1, y_train, epochs=1000, batch_size=16, validation_split=0.2, callbacks=[es])

Epoch 1/1000


2023-12-12 14:42:36.564759: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:219] failed to create cublas handle: cublas error
2023-12-12 14:42:36.564810: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:221] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2023-12-12 14:42:36.564837: W tensorflow/core/framework/op_kernel.cc:1830] OP_REQUIRES failed at matmul_op_impl.h:622 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Graph execution error:

Detected at node 'sequential_2/dense_4/MatMul' defined at (most recent call last):
    File "/opt/conda/envs/tensorflow/lib/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/opt/conda/envs/tensorflow/lib/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/opt/conda/envs/tensorflow/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/opt/conda/envs/tensorflow/lib/python3.10/site-packages/traitlets/config/application.py", line 1053, in launch_instance
      app.start()
    File "/opt/conda/envs/tensorflow/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 737, in start
      self.io_loop.start()
    File "/opt/conda/envs/tensorflow/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 195, in start
      self.asyncio_loop.run_forever()
    File "/opt/conda/envs/tensorflow/

In [ ]:
textcnnmodel = Sequential()
 textcnnmodel.add(layers.Embedding(vocab_size, embedding_dim, input_length=maxlen))
 textcnnmodel.add(layers.)
 textcnnmodel.add(layers.GlobalMaxPooling1D())
 textcnnmodel.add(layers.Dense(10, activation='relu'))
 textcnnmodel.add(layers.Dense(1, activation='sigmoid'))
 textcnnmodel.compile(optimizer='adam',
               loss='binary_crossentropy',
               metrics=['accuracy'])
 textcnnmodel.summary() 